## Exploração de dados

Link do dataset no Kaggle: **https://www.kaggle.com/datasets/shreyanshverma27/water-quality-testing/data**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dados = pd.read_csv("Water Quality Testing.csv")
dados

### Dropando coluna ID

In [ ]:
dados_dropado = dados.drop(columns=["ID Amostra"])
dados_dropado


### Criando a coluna "condicao_animais_marinhos"

Para determinar se as condições da água estão prejudiciais para os animais marinhos com base nos dados fornecidos (pH, Temperatura, Turbidez, Oxigênio Dissolvido e Condutividade), podemos estabelecer critérios gerais com base em padrões aceitos para esses parâmetros. Aqui está um exemplo de como você pode criar uma coluna no seu dataframe para indicar se as condições são prejudiciais ou não:

Definir Critérios de Qualidade da Água:

Os critérios podem variar dependendo da região e do tipo de animais marinhos em questão. Vou usar valores genéricos para ilustração.
Exemplo de Critérios:

pH: Geralmente, valores fora da faixa de 6.5 a 8.5 podem ser prejudiciais.

Temperatura: Varia dependendo das espécies, mas temperatura acima de 30°C será considerada prejudicial..

Turbidez: Valores elevados indicam alta quantidade de partículas em suspensão, turbidez acima de 20 NTU será considerada prejudicial.

Oxigênio Dissolvido: Valores abaixo de 4 mg/L podem ser prejudiciais para muitas espécies.

Condutividade: Valores muito altos podem indicar alta concentração de sais, condutividade acima de 1000 µS/cm será considerada prejudicial.


In [ ]:

df = pd.DataFrame(dados_dropado)

# Função para verificar se as condições são prejudiciais
def condição_prejudicial(row):
    # Critérios de qualidade da água (valores genéricos)
    crit_pH = (row['pH'] < 6.5 or row['pH'] > 8.5)
    crit_OD = (row['Oxigênio Dissolvido (mg/L)'] < 4.0)
    crit_Turbidez = (row['Turbidez (NTU)'] > 20)  # Exemplo de critério para turbidez
    crit_Condutividade = (row['Condutividade (µS/cm)'] > 1000)  # Exemplo de critério para condutividade
    crit_Temperatura = (row['Temperatura (°C)'] > 30)  # Exemplo de critério para temperatura

    # Verifica se pelo menos um critério é verdadeiro
    if crit_pH or crit_OD or crit_Turbidez or crit_Condutividade or crit_Temperatura:
        return 1
    else:
        return 0


### Gerando dados aleatórios para coluna "condicao_animais_marinhos" ser prejudicial para o treinamento da IA e add ao df

In [ ]:
# Gerar 200 dados prejudiciais aleatórios
np.random.seed(0)  # Seed para reprodutibilidade
n_prejudiciais = 350

# Gerar valores aleatórios
pH_prejudicial = np.random.uniform(low=4.0, high=9.0, size=n_prejudiciais)
temperatura_prejudicial = np.random.uniform(low=20.0, high=35.0, size=n_prejudiciais)
turbidez_prejudicial = np.random.uniform(low=20.0, high=50.0, size=n_prejudiciais)
OD_prejudicial = np.random.uniform(low=2.0, high=3.9, size=n_prejudiciais)
condutividade_prejudicial = np.random.uniform(low=800, high=1500, size=n_prejudiciais)

# Criar DataFrame com dados prejudiciais
novos_dados = {
    'pH': pH_prejudicial,
    'Temperatura (°C)': temperatura_prejudicial,
    'Turbidez (NTU)': turbidez_prejudicial,
    'Oxigênio Dissolvido (mg/L)': OD_prejudicial,
    'Condutividade (µS/cm)': condutividade_prejudicial,
    'condicao_animais_marinhos': ['Prejudicial'] * n_prejudiciais
}

df_novos = pd.DataFrame(novos_dados)

# Adicionar os novos dados ao DataFrame original
df = pd.concat([df, df_novos], ignore_index=True)

# Aplicar a função para verificar as condições
df['condicao_animais_marinhos'] = df.apply(condição_prejudicial, axis=1)


In [ ]:
# Visualizar os primeiros registros para verificar
dados_mar = df
dados_mar.head()

In [ ]:
# Visualizar os ultimos registros para verificar
dados_mar.tail()

## Treinamento do modelo KNN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Carregar o dataset
df = dados_mar

# Separar features (X) e target (y)
X = df[['pH', 'Temperatura (°C)', 'Turbidez (NTU)', 'Oxigênio Dissolvido (mg/L)', 'Condutividade (µS/cm)']]
y = df['condicao_animais_marinhos']  # Usaremos o evento prejudicial combinado como target

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Função de normalização personalizada
def normalizar(x, min_x, max_x):
    return (x - min_x) / (max_x - min_x)

# Calcular mínimos e máximos dos dados de treino
min_train = X_train.min()
max_train = X_train.max()

# Aplicar a função de normalização aos conjuntos de treino e teste
X_train_normalizado = normalizar(X_train, min_train, max_train)
X_test_normalizado = normalizar(X_test, min_train, max_train)

# Criar um modelo KNN
knn = KNeighborsClassifier(n_neighbors=5)

# Treinar o modelo
knn.fit(X_train_normalizado, y_train)

# Fazer previsões no conjunto de teste
y_pred = knn.predict(X_test_normalizado)

# Avaliar o desempenho do modelo
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

In [ ]:
# Exemplos de novos dados para prever
novos_dados = [
    [7.0, 25.0, 14.0, 5.7, 500.0], # Não prejudicial
    [6.5, 22.0, 12.0, 6.5, 450.0], # Não prejudicial
    [8.0, 26.0, 16.0, 5.0, 2000],  # Prejudicial
    [7.2, 24.0, 15.0, 6.0, 480.0], # Não prejudical
    [8.0, 31.0, 22.0, 5.0, 2000]   # Prejudicial
]

# Converter novos dados para um DataFrame para aplicar a normalização
novos_dados_df = pd.DataFrame(novos_dados, columns=['pH', 'Temperatura (°C)', 'Turbidez (NTU)', 'Oxigênio Dissolvido (mg/L)', 'Condutividade (µS/cm)'])

# Normalizar os novos dados usando os mínimos e máximos dos dados de treino
novos_dados_norm = normalizar(novos_dados_df, min_train, max_train)

# Prever se será um evento prejudicial ou não
previsoes = knn.predict(novos_dados_norm)

print("\nPrevisões para os novos dados:")
for i, pred in enumerate(previsoes):
    print(f"Exemplo {i+1}: {'Será um evento prejudicial' if pred == 1 else 'Não será um evento prejudicial'}")